# pyLectureMultiModalAnalysis

## Summary of video segments

### Functions: 3
#### add_duration(), labels2summary(),summary2video()

### Author: Stelios Karozis

In [2]:
import pandas as pd
import numpy as np

df=pd.read_csv('../data/index.csv', sep=';')
df

,FILE,SEG,CLASS_1
0,medium_0,part_0,boring
1,medium_0,part_1,interesting
2,medium_0,part_2,boring
3,medium_0,part_3,interesting
4,medium_0,part_4,interesting
...,...,...,...
245,medium_7,part_45,interesting
246,medium_7,part_46,interesting
247,medium_7,part_47,neutral
248,medium_7,part_48,interesting


In [13]:
def add_duration(df=df, folder='../data/'):
    from pymediainfo import MediaInfo
    dur=[]
    for index,f in df.iterrows():
        #print(f['FILE'])
        media_info = MediaInfo.parse(folder+f['FILE']+'/'+f['SEG']+'.mp4')
    #duration in milliseconds
        duration_in_s = media_info.tracks[0].duration/1000
        dur.append(duration_in_s)
    df['DURATION']=dur
    return df

In [440]:
#add_duration(df=df)

,FILE,SEG,CLASS_1,DURATION
0,medium_0,part_0,boring,17.040
1,medium_7,part_28,boring,21.479
2,medium_0,part_1,interesting,18.716
3,medium_0,part_2,boring,23.337
4,medium_0,part_3,interesting,21.850
...,...,...,...,...
249,medium_7,part_46,interesting,18.739
250,medium_7,part_47,neutral,22.524
251,medium_7,part_48,interesting,18.437
252,medium_7,part_49,interesting,20.755


In [4]:
def labels2summary(df=df, col_cl='CLASS_1', label=['boring','interesting','neutral'], prc_l=[0.30,0.35,0.35], col_d='DURATION', duration_sec=90):
    summary={}
    i=0
    for l,p in zip(label, prc_l):
        df_label=df.loc[df[col_cl] == l]
        
        #In order to meet the total duration of summary we use the avg duration of each class
        avg_dur=round(df_label[col_d].mean())
        
        #We calculate the seconds correspond to each class
        #and then how many parts of each class (of avg duration) 
        #need to be used for each time span 
        if duration_sec != -1:
            ndx=int(round((p*duration_sec/avg_dur)))
        else:
            ndx=int(len(df_label[col_cl])*p)
            
        tmp=df_label[['FILE','SEG',col_cl]].iloc[0:ndx]

        i=i+1
        if i == 1:
            summary=tmp
        else:
            summary=pd.concat([summary,tmp])
    return summary.reset_index().set_index([col_cl,'FILE'])

In [462]:
#summary= labels2summary(df=df, col_cl='CLASS_1', label=['boring','interesting','neutral'], prc_l=[0.30,0.35,0.35], col_d='DURATION', duration_sec=360)
#summary

index      SEG
CLASS_1     FILE                    
boring      medium_0      0   part_0
            medium_7      1  part_28
            medium_0      3   part_2
            medium_0     11  part_10
            medium_0     12  part_11
interesting medium_0      2   part_1
            medium_0      4   part_3
            medium_0      5   part_4
            medium_0      6   part_5
            medium_0      7   part_6
            medium_0     10   part_9
neutral     medium_0      8   part_7
            medium_0      9   part_8
            medium_0     13  part_12
            medium_0     16  part_15
            medium_0     18  part_17
            medium_0     21  part_20

In [15]:
def summary2video(df, output_folder='../data/', output_name='summary.mp4'):
    import os
    from pymediainfo import MediaInfo
    from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip
    
    try:
        os.remove(output_name)
    except OSError:
        pass
    clips=[]
    for i, new_df in df.groupby(level=0):
        for folder,file in df.loc[i].iterrows():
            print(folder,file['SEG'])
            clip = VideoFileClip(output_folder+folder+'/'+file['SEG']+'.mp4')
            clips.append(clip)
    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile(output_folder+output_name)
    
    media_info = MediaInfo.parse(output_folder+output_name)
    duration_in_s = media_info.tracks[0].duration/1000
    print('Summary duration is:', duration_in_s)

In [486]:
#summary2video(df=summary)

medium_0 part_0
medium_7 part_28
medium_0 part_2
medium_0 part_10
medium_0 part_11
medium_0 part_1
medium_0 part_3
medium_0 part_4
medium_0 part_5
medium_0 part_6
medium_0 part_9
medium_0 part_7
medium_0 part_8
medium_0 part_12
medium_0 part_15
medium_0 part_17
medium_0 part_20


chunk:   1%|          | 84/7740 [00:00<00:09, 838.71it/s, now=None]

Moviepy - Building video ../data/summary.mp4.
MoviePy - Writing audio in summaryTEMP_MPY_wvf_snd.mp3


t:   1%|          | 80/10520 [00:00<00:13, 784.55it/s, now=None]      

MoviePy - Done.
Moviepy - Writing video ../data/summary.mp4



Moviepy - Done !
Moviepy - video ready ../data/summary.mp4
Summary duration is: 351.06


In [11]:
def df2summary(df=df, folderDATA='../data/' ,col_cl='CLASS_1', label=['boring','interesting','neutral'], prc_l=[0.30,0.35,0.35], col_d='DURATION', duration_sec=90, output_folder= '../data/', output_name='summary.mp4'):

    df_dur=add_duration(df=df, folder=folderDATA)

    summary=labels2summary(df=df_dur, col_cl=col_cl, label=label, prc_l=prc_l, col_d=col_d, duration_sec=duration_sec)

    summary2video(df=summary, output_folder=output_folder, output_name=output_name)
    
    return print('Enjoy your summary video !')

In [16]:
df2summary(df=df, folderDATA='../data/' ,col_cl='CLASS_1', label=['boring','interesting','neutral'], prc_l=[0.30,0.35,0.35], col_d='DURATION', duration_sec=90, output_folder= '../data/', output_name='summary.mp4')

medium_0 part_0
medium_0 part_1
medium_0 part_7


chunk:   9%|▉         | 123/1300 [00:00<00:00, 1225.81it/s, now=None]

Moviepy - Building video ../data/summary.mp4.
MoviePy - Writing audio in summaryTEMP_MPY_wvf_snd.mp3


t:   5%|▌         | 77/1474 [00:00<00:01, 765.63it/s, now=None]       

MoviePy - Done.
Moviepy - Writing video ../data/summary.mp4



Moviepy - Done !
Moviepy - video ready ../data/summary.mp4
Summary duration is: 58.96
Enjoy your summary video !
